# Извлечение именованных сущностей (Named-entity recognition, NER)

+ Задача NER – выделить спаны именованных сущностей в тексте
+ Изначально, именованные сущности -- это персоны, локации, организации
+ Обычно типов больше: даты, денежные суммы, прочее (например, названия брендов)
+ Зачем? Решать задачи референции, референциального выбора и кореференции, метонимии, которые являются центральными для поиска, вопросно-ответных систем, связности текста, синтаксического и морфологического парсинга и т.д.
+ Сложности:
    - омонимия: "Вашингтон" -- город, штат, фамилия, имя жирафа, название компании?
    - технические: какие теги? где границы сущности?
+ BIOES-схема: к метке сущности (например, PER для персон или ORG для организаций) добавляется префикс, который обозначает позицию токена в спане сущности:

  - B – beginning – первый токен в спане сущности, которая состоит из нескольких токенов;
  - I – inside – внутри спана;
  - О – outside – токен не относится ни к какой сущности;
  - E – ending – последний токен сущности, которая состоит из нескольких токенов;
  - S – single – сущность состоит из одного токена.

In [13]:
ru_text = '''Вторая нитка "Северного потока - 2" заполнена газом, и теперь газопровод полностью готов к эксплуатации. Об этом доложил в среду глава "Газпрома" Алексей Миллер президенту РФ Владимиру Путину.

Однако это не означает, что газопровод будет запущен в ближайшие дни и даже месяцы, - ранее вице-премьер Александр Новак выражал надежду, что его сертификация завершится к концу первой половины 2022 года. В самой Германии заявляли, что соответствующее решение не будет принято в первом полугодии. '''

In [9]:
eng_text = '''MOSCOW, Dec 29 (Reuters) - Russian President Vladimir Putin said on Wednesday the Nord Stream 2 undersea gas pipeline would help to calm a surge in European gas prices and was ready to start exports now a second stretch of the pipeline has been filled.

Nord Stream 2, completed in September but awaiting regulatory approval from Germany and the European Union, faces resistance from the United States and several countries including Poland and Ukraine, which say it will increase Russia's leverage over Europe.

The pipeline had been scheduled to be completed in 2019, but construction was suspended following the threat of sanctions by the U.S. administration of Donald Trump and the subsequent withdrawal of the Swiss-Dutch company Allseas from pipe-laying.'''

In [34]:
de_text = '''Der neu gewählte Bundeskanzler Olaf Scholz (SPD) traf am Freitag in Brüssel mit Kommissionspräsidentin Ursula von der Leyen zusammen. Beide richteten warnende Worte an Russland. Von der Leyen sagte: „Wir erwarten, dass Russland deeskaliert und jegliche Art von Aggression gegenüber seinen Nachbarn unterlässt und die Rechte souveräner Staaten achtet. Andernfalls ist die Europäische Union bereit, nicht nur die bestehenden Sanktionen zu verschärfen, sondern auch neue, spürbare Maßnahmen zu ergreifen.“ Die Kommissionspräsidentin nannte die Felder Wirtschaft und Finanzen. '''

## Stanza

+ https://stanfordnlp.github.io/stanza/
+ библиотека для NLP
+ 66 языков, включая русский
+ токенизация, лемматизация, морфологический и синтаксический парсинг, NER

In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('en')

Для выявления именованных сущностей необходимо токенизировать текст.

In [17]:
def stanza_nlp(text):
  nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
  doc = nlp(text)
  print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

In [18]:
stanza_nlp(eng_text)

2021-12-30 12:18:54 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-12-30 12:18:54 INFO: Use device: cpu
2021-12-30 12:18:54 INFO: Loading: tokenize
2021-12-30 12:18:54 INFO: Loading: ner
2021-12-30 12:18:55 INFO: Done loading processors!


entity: MOSCOW	type: GPE
entity: Dec 29 (Reuters) -	type: DATE
entity: Russian	type: NORP
entity: Vladimir Putin	type: PERSON
entity: Wednesday	type: DATE
entity: 2	type: CARDINAL
entity: European	type: NORP
entity: second	type: ORDINAL
entity: Nord Stream 2	type: ORG
entity: September	type: DATE
entity: Germany	type: GPE
entity: the European Union	type: ORG
entity: the United States	type: GPE
entity: Poland	type: GPE
entity: Ukraine	type: GPE
entity: Russia	type: GPE
entity: Europe	type: LOC
entity: 2019	type: DATE
entity: U.S.	type: GPE
entity: Donald Trump	type: PERSON
entity: Swiss	type: NORP
entity: Allseas	type: ORG


Если нужны BIOES NER теги для каждого токена:

In [33]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
doc = nlp(eng_text)
print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens][10:50], sep='\n')

2021-12-30 12:39:50 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-12-30 12:39:50 INFO: Use device: cpu
2021-12-30 12:39:50 INFO: Loading: tokenize
2021-12-30 12:39:50 INFO: Loading: ner
2021-12-30 12:39:51 INFO: Done loading processors!


token: Vladimir	ner: B-PERSON
token: Putin	ner: E-PERSON
token: said	ner: O
token: on	ner: O
token: Wednesday	ner: S-DATE
token: the	ner: O
token: Nord	ner: O
token: Stream	ner: O
token: 2	ner: S-CARDINAL
token: undersea	ner: O
token: gas	ner: O
token: pipeline	ner: O
token: would	ner: O
token: help	ner: O
token: to	ner: O
token: calm	ner: O
token: a	ner: O
token: surge	ner: O
token: in	ner: O
token: European	ner: S-NORP
token: gas	ner: O
token: prices	ner: O
token: and	ner: O
token: was	ner: O
token: ready	ner: O
token: to	ner: O
token: start	ner: O
token: exports	ner: O
token: now	ner: O
token: a	ner: O
token: second	ner: S-ORDINAL
token: stretch	ner: O
token: of	ner: O
token: the	ner: O
token: pipeline	ner: O
token: has	ner: O
token: been	ner: O
token: filled	ner: O
token: .	ner: O
token: Nord	ner: B-ORG


Протестируем для русского языка:

In [ ]:
stanza.download('ru')

In [12]:
def stanza_nlp_ru(text):
  nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')
  doc = nlp(text)
  print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

In [14]:
stanza_nlp_ru(ru_text)

2021-12-30 12:04:00 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2021-12-30 12:04:00 INFO: Use device: cpu
2021-12-30 12:04:00 INFO: Loading: tokenize
2021-12-30 12:04:00 INFO: Loading: ner
2021-12-30 12:04:01 INFO: Done loading processors!


entity: Северного потока - 2	type: MISC
entity: Газпрома	type: ORG
entity: Алексей Миллер	type: PER
entity: РФ	type: LOC
entity: Владимиру Путину	type: PER
entity: Александр Новак	type: PER
entity: Германии	type: LOC


Выберите интересный Вам язык и протестируйте работу stanza на нем, какие проблемы выявления именованных сущностей специфичны для выбранного Вами языка?

In [36]:
def stanza_nlp_de(text):
  nlp = stanza.Pipeline(lang='de', processors='tokenize,ner')
  doc = nlp(text)
  print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')
stanza_nlp_de(de_text)

2021-12-30 12:49:40 WARNING: Language de package default expects mwt, which has been added
2021-12-30 12:49:40 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| ner       | conll03 |

2021-12-30 12:49:40 INFO: Use device: cpu
2021-12-30 12:49:40 INFO: Loading: tokenize
2021-12-30 12:49:40 INFO: Loading: mwt
2021-12-30 12:49:40 INFO: Loading: ner
2021-12-30 12:49:42 INFO: Done loading processors!
/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


entity: Olaf Scholz	type: PER
entity: SPD	type: ORG
entity: Brüssel	type: LOC
entity: Ursula von der Leyen	type: PER
entity: Russland	type: LOC
entity: Leyen	type: PER
entity: Russland	type: LOC
entity: Europäische Union	type: ORG


## SpaCy

+ Библиотека для продвинутого NLP
+ Ряд языков, английский, китайский, немецкий, французский, итальянский, польский, испанский и др., разрабатываются модели для всё новых языков
+ Про spaCy: https://spacy.io/usage

+ Установка для английского:

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [38]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [39]:
doc = nlp(eng_text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

MOSCOW 0 6 GPE
Dec 29 8 14 DATE
Reuters 16 23 ORG
Russian 27 34 NORP
Vladimir Putin 45 59 PERSON
Wednesday 68 77 DATE
2 94 95 CARDINAL
European 148 156 NORP
second 205 211 ORDINAL
September 282 291 DATE
Germany 330 337 GPE
the European Union 342 360 ORG
the United States 384 401 GPE
Poland 434 440 GPE
Ukraine 445 452 GPE
Russia 481 487 GPE
Europe 504 510 LOC
2019 564 568 DATE
U.S. 642 646 GPE
Donald Trump 665 677 PERSON
Swiss 715 720 NORP


SpaCy предлагает 18 тегов (список можно посмотреть [здесь](https://towardsdatascience.com/named-entity-recognition-ner-using-spacy-nlp-part-4-28da2ece57c6)) для разных типов именованных сущностей, также легко можно добавить новый тег:

In [43]:
from spacy.tokens import Span

nlp = spacy.load("en_core_web_sm")
doc = nlp(eng_text)
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('Before', ents)
# the model didn't recognize "Nord Stream" as an entity :(

ns2_ent = Span(doc, 16, 18, label="OBJ") # create a Span for the new entity, OBJ = object
doc.ents = list(doc.ents) + [ns2_ent]

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('After', ents)
#('Nord Stream', 82, 93, 'OBJ') got recognized as an object entity


Before [('MOSCOW', 0, 6, 'GPE'), ('Dec 29', 8, 14, 'DATE'), ('Reuters', 16, 23, 'ORG'), ('Russian', 27, 34, 'NORP'), ('Vladimir Putin', 45, 59, 'PERSON'), ('Wednesday', 68, 77, 'DATE'), ('2', 94, 95, 'CARDINAL'), ('European', 148, 156, 'NORP'), ('second', 205, 211, 'ORDINAL'), ('September', 282, 291, 'DATE'), ('Germany', 330, 337, 'GPE'), ('the European Union', 342, 360, 'ORG'), ('the United States', 384, 401, 'GPE'), ('Poland', 434, 440, 'GPE'), ('Ukraine', 445, 452, 'GPE'), ('Russia', 481, 487, 'GPE'), ('Europe', 504, 510, 'LOC'), ('2019', 564, 568, 'DATE'), ('U.S.', 642, 646, 'GPE'), ('Donald Trump', 665, 677, 'PERSON'), ('Swiss', 715, 720, 'NORP')]
After [('MOSCOW', 0, 6, 'GPE'), ('Dec 29', 8, 14, 'DATE'), ('Reuters', 16, 23, 'ORG'), ('Russian', 27, 34, 'NORP'), ('Vladimir Putin', 45, 59, 'PERSON'), ('Wednesday', 68, 77, 'DATE'), ('Nord Stream', 82, 93, 'OBJ'), ('2', 94, 95, 'CARDINAL'), ('European', 148, 156, 'NORP'), ('second', 205, 211, 'ORDINAL'), ('September', 282, 291, 'DATE'

In [45]:
len([ent for ent in doc.ents if ent.label_=='OBJ'])

1

In [47]:
from spacy import displacy

In [49]:
displacy.render(doc,style = "ent",jupyter=True)

Протестируем для русского языка:

In [ ]:
! python -m spacy download ru_core_news_sm

In [51]:
nlp = spacy.load("ru_core_news_sm")

In [52]:
doc = nlp(ru_text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Газпрома 136 144 ORG
Алексей Миллер 146 160 PER
РФ 172 174 LOC
Владимиру Путину 175 191 PER
Александр Новак 299 314 PER
Германии 407 415 LOC


In [53]:
displacy.render(doc, style = "ent", jupyter=True)

Можно подкрасить только сущности определенных типов:

In [54]:
options = {'ents':['ORG','LOC']}
displacy.render(doc, style = "ent", jupyter=True, options=options)


Как SpaCy справляется с выбранным Вами языком? Возникают ли проблемы с распознаванием именованных сущностей? 

In [ ]:
! python -m spacy download de_core_news_sm

In [56]:
nlp = spacy.load("de_core_news_sm")

In [57]:
doc = nlp(de_text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Olaf Scholz 31 42 PER
Brüssel 68 75 LOC
Ursula 103 109 PER
Leyen 118 123 LOC
Russland. 168 177 MISC
Leyen 186 191 LOC
Russland 219 227 LOC
Staaten 335 342 LOC
Europäische Union 371 388 ORG
Die Kommissionspräsidentin 503 529 MISC


## Natasha

+ Раньше библиотека Natasha решала задачу NER для русского языка, была построена на правилах
+ сейчас это полноценный NLP проект для русского языка
+ токенизация, лемматизация, синтаксический разбор, NER-тегирование и т.д.
+ https://github.com/natasha/natasha


In [ ]:
!pip install natasha

In [59]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

In [60]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
doc = Doc(ru_text)

NER зависит от сегментации:

In [62]:
doc.segment(segmenter)
print(doc.tokens[:5])
print(doc.sents[:5])

[DocToken(stop=6, text='Вторая'), DocToken(start=7, stop=12, text='нитка'), DocToken(start=13, stop=14, text='"'), DocToken(start=14, stop=23, text='Северного'), DocToken(start=24, stop=30, text='потока')]
[DocSent(stop=104, text='Вторая нитка "Северного потока - 2" заполнена газ..., tokens=[...]), DocSent(start=105, stop=192, text='Об этом доложил в среду глава "Газпрома" Алексей ..., tokens=[...]), DocSent(start=194, stop=398, text='Однако это не означает, что газопровод будет запу..., tokens=[...]), DocSent(start=399, stop=490, text='В самой Германии заявляли, что соответствующее ре..., tokens=[...])]


In [67]:
doc.tag_ner(ner_tagger)
display(doc.spans)
doc.ner.print()

[DocSpan(start=136, stop=144, type='ORG', text='Газпрома', tokens=[...]),
 DocSpan(start=146, stop=160, type='PER', text='Алексей Миллер', tokens=[...]),
 DocSpan(start=172, stop=174, type='LOC', text='РФ', tokens=[...]),
 DocSpan(start=175, stop=191, type='PER', text='Владимиру Путину', tokens=[...]),
 DocSpan(start=299, stop=314, type='PER', text='Александр Новак', tokens=[...]),
 DocSpan(start=407, stop=415, type='LOC', text='Германии', tokens=[...])]

Вторая нитка "Северного потока - 2" заполнена газом, и теперь 
газопровод полностью готов к эксплуатации. Об этом доложил в среду 
глава "Газпрома" Алексей Миллер президенту РФ Владимиру Путину.
       ORG─────  PER───────────            LO PER───────────── 
Однако это не означает, что газопровод будет запущен в ближайшие дни и
 даже месяцы, - ранее вице-премьер Александр Новак выражал надежду, 
                                   PER────────────                  
что его сертификация завершится к концу первой половины 2022 года. В 
самой Германии заявляли, что соответствующее решение не будет принято 
      LOC─────                                                        
в первом полугодии. 


Можно привести сущности к нормальной форме, для этого надо провести морфологический анализ и лемматизацию (Natasha использует Pymorphy2):

In [71]:
doc.tag_morph(morph_tagger)
doc.sents[0].morph.print()

              Вторая ADJ|Case=Nom|Degree=Pos|Gender=Fem|Number=Sing
               нитка NOUN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing
                   " PUNCT
           Северного ADJ|Case=Gen|Degree=Pos|Gender=Masc|Number=Sing
              потока NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
                   - PUNCT
                   2 NUM
                   " PUNCT
           заполнена VERB|Aspect=Perf|Gender=Fem|Number=Sing|Tense=Past|Variant=Short|VerbForm=Part|Voice=Pass
               газом NOUN|Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing
                   , PUNCT
                   и CCONJ
              теперь ADV|Degree=Pos
          газопровод NOUN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
           полностью ADV|Degree=Pos
               готов ADJ|Degree=Pos|Gender=Masc|Number=Sing|Variant=Short
                   к ADP
        эксплуатации NOUN|Animacy=Inan|Case=Dat|Gender=Fem|Number=Sing
                   . PUNCT


In [72]:
for token in doc.tokens:
  token.lemmatize(morph_vocab)
    
{_.text: _.lemma for _ in doc.tokens}

{'"': '"',
 ',': ',',
 '-': '-',
 '.': '.',
 '2': '2',
 '2022': '2022',
 'Александр': 'александр',
 'Алексей': 'алексей',
 'В': 'в',
 'Владимиру': 'владимир',
 'Вторая': 'второй',
 'Газпрома': 'газпром',
 'Германии': 'германия',
 'Миллер': 'миллер',
 'Новак': 'новак',
 'Об': 'о',
 'Однако': 'однако',
 'Путину': 'путин',
 'РФ': 'рф',
 'Северного': 'северный',
 'ближайшие': 'близкий',
 'будет': 'быть',
 'в': 'в',
 'вице-премьер': 'вице-премьер',
 'выражал': 'выражать',
 'газом': 'газ',
 'газопровод': 'газопровод',
 'глава': 'глава',
 'года': 'год',
 'готов': 'готовый',
 'даже': 'даже',
 'дни': 'день',
 'доложил': 'доложить',
 'его': 'его',
 'завершится': 'завершиться',
 'заполнена': 'заполнить',
 'запущен': 'запустить',
 'заявляли': 'заявлять',
 'и': 'и',
 'к': 'к',
 'концу': 'конец',
 'месяцы': 'месяц',
 'надежду': 'надежда',
 'не': 'не',
 'нитка': 'нитка',
 'означает': 'означать',
 'первой': 'первый',
 'первом': 'первый',
 'полностью': 'полностью',
 'половины': 'половина',
 'полугодии'

Приводим сущности к нормальной форме:

In [73]:
for span in doc.spans:
  span.normalize(morph_vocab)

{_.text: _.normal for _ in doc.spans if _.text != _.normal}

{'Владимиру Путину': 'Владимир Путин',
 'Газпрома': 'Газпром',
 'Германии': 'Германия'}

Можно извлечь для нормированных имен отдельно имена и фамилии:

In [78]:
for span in doc.spans:
  if span.type == PER:
    span.extract_fact(names_extractor)

{_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}

{'Александр Новак': {'first': 'Александр'},
 'Алексей Миллер': {'first': 'Алексей', 'last': 'Миллер'},
 'Владимир Путин': {'first': 'Владимир', 'last': 'Путин'}}

In [82]:
#names_extractor = NamesExtractor(morph_vocab)
#dates_extractor = DatesExtractor(morph_vocab)
#money_extractor = MoneyExtractor(morph_vocab)
#addr_extractor = AddrExtractor(morph_vocab)